# Recurrent Networks in Deep Learning

1. Introduction
2. Long Short-Term Memory model
3. LSTM
4. Stacked LSTM

### 1. Introduction

Recurrent Neural Networks are Deep Learning models with simple structures and a feedback mechanism built-in, or in different words, the output of a layer is added to the next input and fed back to the same layer.

The Recurrent Neural Network is a specialized type of Neural Network that solves the issue of maintaining context for Sequential data -- such as Weather data, Stocks, Genes, etc. At each iterative step, the processing unit takes in an input and the current state of the network, and produces an output and a new state that is re-fed into the network.


<img src="https://ibm.box.com/shared/static/v7p90neiaqghmpwawpiecmz9n7080m59.png">

However, this model has some problems. It's very computationally expensive to maintain the state for a large amount of units, even more so over a long amount of time. Additionally, Recurrent Networks are very sensitive to changes in their parameters. As such, they are prone to different problems with their Gradient Descent optimizer -- they either grow exponentially (Exploding Gradient) or drop down to near zero and stabilize (Vanishing Gradient), both problems that greatly harm a model's learning capability.

To solve these problems, Hochreiter and Schmidhuber published a paper in 1997 describing a way to keep information over long periods of time and additionally solve the oversensitivity to parameter changes, i.e., make backpropagating through the Recurrent Networks more viable. This proposed method is called Long Short-Term Memory (LSTM).

### 2. Long Short Term Memory Model

The Long Short-Term Memory, as it was called, was an abstraction of how computer memory works. It is "bundled" with whatever processing unit is implemented in the Recurrent Network, although outside of its flow, and is responsible for keeping, reading, and outputting information for the model. 

The way it works is simple: you have a linear unit, which is the information cell itself, surrounded by three logistic gates responsible for maintaining the data. One gate is for inputting data into the information cell, one is for outputting data from the input cell, and the last one is to keep or forget data depending on the needs of the network.

Thanks to that, it not only solves the problem of keeping states, because the network can choose to forget data whenever information is not needed, it also solves the gradient problems, since the Logistic Gates have a very nice derivative.

**Long short-Term Memory architecture**

The Long Short-Term Memory is composed of a linear unit surrounded by three logistic gates. The name for these gates vary from place to place, but the most usual names for them are:

* the "Input" or "Write" Gate, which handles the writing of data into the information cell
* the "Output" or "Read" Gate, which handles the sending of data back onto the Recurrent Network
* the "Keep" or "Forget" Gate, which handles the maintaining and modification of the data stored in the information cell

<img src="https://ibm.box.com/shared/static/zx10duv5egw0baw6gh2hzsgr8ex45gsg.png" width="720"/>

The three gates are the centerpiece of the LSTM unit. The gates, when activated by the network, perform their respective functions. For example, the Input Gate will write whatever data it is passed into the information cell, the Output Gate will return whatever data is in the information cell, and the Keep Gate will maintain the data in the information cell. These gates are analog and multiplicative, and as such, can modify the data based on the signal they are sent.


For example, an usual flow of operations for the LSTM unit is as such: First off, the Keep Gate has to decide whether to keep or forget the data currently stored in memory. It receives both the input and the state of the Recurrent Network, and passes it through its Sigmoid activation. If  $𝐾_t$  has value of 1 means that the LSTM unit should keep the data stored perfectly and if  $𝐾_t$   a value of 0 means that it should forget it entirely. Consider  $S_{t-1}$   as the incoming (previous) state,  $𝑥_𝑡$  as the incoming input, and  $W_k$ , $B_k$  as the weight and bias for the Keep Gate. Additionally, consider  $Old_{t-1}$  as the data previously in memory. What happens can be summarized by this equation:

$k_t = σ (W_k × [S_{t-1}, x_t] + B_k)$

$New_t = I_t \times C_t$

$New_t$  is the new data to be input into the memory cell. This is then added to whatever value is still stored in memory.

$Cell_t = Old_t + New_t$

We now have the candidate data which is to be kept in the memory cell. The conjunction of the Keep and Input gates work in an analog manner, making it so that it is possible to keep part of the old data and add only part of the new data. Consider however, what would happen if the Forget Gate was set to 0 and the Input Gate was set to 1:

$Old_t = 0 \times Old_{t-1}$

$New_t = 1 \times C_t$

$Cell_t = C_t$

The old data would be totally forgotten and the new data would overwrite it completely.
The Output Gate functions in a similar manner. To decide what we should output, we take the input data and state and pass it through a Sigmoid function as usual. The contents of our memory cell, however, are pushed onto a **Tanh function** to bind them between a value of -1 to 1. Consider  $W_0$  and  $B_0$  as the weight and bias for the Output Gate.

$O_t = σ (W_o × [S_{t-1}, x_t] + B_o)$

$Output_t = O_t \times tanh(Cell_t)$

And that  $Output_t$  is what is output into the Recurrent Network.


As mentioned many times, all three gates are **logistic**. The reason for this is because it is very easy to backpropagate through them, and as such, it is possible for the model to learn exactly _how_ it is supposed to use this structure. This is one of the reasons for which LSTM is a very strong structure. Additionally, this solves the gradient problems by being able to manipulate values through the gates themselves -- by passing the inputs and outputs through the gates, we have now a easily derivable function modifying our inputs.

In regards to the problem of storing many states over a long period of time, LSTM handles this perfectly by only keeping whatever information is necessary and forgetting it whenever it is not needed anymore. Therefore, LSTMs are a very elegant solution to both problems.

### 3. LSTM

Lets first create a tiny LSTM network sample to understand the architecture of LSTM networks.
We need to import the necessary modules for our code. We need `numpy` and `tensorflow`, obviously. Additionally, we can import directly the `tensorflow.contrib.rnn` model, which includes the function for building RNNs.

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import tensorflow.compat.v1 as tf
sess = tf.Session()

We want to create a network that has only one LSTM cell. We have to pass 2 elements to LSTM, the `prv_output` (**previous output**) and `prv_state` (**previous state**), so called, **h** and **c**. Therefore, we initialize a state vector, state. Here, state is a tuple with 2 elements, each one is of size [1 x 4], one for passing prv_output to next time step, and another for passing the prv_state to next time stamp.

Output size (dimension), which is same as hidden size in the cell

In [ ]:
LSTM_CELL_SIZE = 4 

lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(LSTM_CELL_SIZE, state_is_tuple = True)
state = (tf.zeros([1, LSTM_CELL_SIZE]), ) * 2
state

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  This is separate from the ipykernel package so we can avoid doing imports until


(<tf.Tensor 'zeros_7:0' shape=(1, 4) dtype=float32>,
 <tf.Tensor 'zeros_7:0' shape=(1, 4) dtype=float32>)

Let define a sample input. In this example, batch_size = 1, and seq_len = 6:

In [ ]:
sample_input = tf.constant([[3, 2, 2, 2, 2, 2]], dtype = tf.float32)
print(sess.run(sample_input))

[[3. 2. 2. 2. 2. 2.]]


Now, we can pass the input to lstm_cell, and check the new state:

In [ ]:
with tf.variable_scope("LSTM_sample1"):
    output, state_new = lstm_cell(sample_input, state)

sess.run(tf.global_variables_initializer())
print(sess.run(state_new))

LSTMStateTuple(c=array([[ 0.40205398, -0.07994743, -0.14456598, -0.09809282]],
      dtype=float32), h=array([[ 0.11147372, -0.06176465, -0.09219049, -0.00961837]],
      dtype=float32))


As we can see, the states has 2 parts, the new state c, and also the output h. Lets check the output again:

In [ ]:
print(sess.run(output))

[[ 0.11147372 -0.06176465 -0.09219049 -0.00961837]]


### 4. Stacked LSTM

What about if we want to have a RNN with stacked LSTM? For example, a 2-layer LSTM. In this case, the output of the first layer will become the input of the second.
Lets start with a new session:

In [ ]:
sess = tf.Session()

In [ ]:
input_dim = 6

Lets create the stacked LSTM cell:

In [ ]:
cells = []

Creating the first layer LTSM cell.

In [ ]:
LSTM_CELL_SIZE_1 = 4    #Hidden nodes
cell1 = tf.keras.layers.LSTMCell(LSTM_CELL_SIZE_1)
cells.append(cell1)

Creating the second layer LTSM cell.

In [ ]:
LSTM_CELL_SIZE_2 = 5  
cell2 = tf.keras.layers.LSTMCell(LSTM_CELL_SIZE_2)
cells.append(cell2)

To create a multi-layer LTSM we use the `tf.keras.layers.StackedRNNCells` function, it takes in multiple single layer LTSM cells to create a multilayer stacked LTSM model.

In [ ]:
stacked_lstm = tf.keras.layers.StackedRNNCells(cells)

Now we can create the RNN from stacked_lstm:

In [ ]:
# Batch size x time steps x features.
data = tf.placeholder(tf.float32, [None, None, input_dim])
output, state = tf.nn.dynamic_rnn(stacked_lstm, data, dtype=tf.float32)

Lets say the input sequence length is 3, and the dimensionality of the inputs is 6. The input should be a Tensor of shape: [batch_size, max_time, dimension], in our case it would be (2, 3, 6)

In [ ]:
#Batch size x time steps x features.
sample_input = [[[1, 2, 3, 4, 3, 2], [1, 2, 1, 1, 1, 2], [1, 2, 2, 2, 2, 2]], 
                [[1, 2, 3, 4, 3, 2], [3, 2, 2, 1, 1, 2], [0, 0, 0, 0, 3, 2]]]
sample_input

[[[1, 2, 3, 4, 3, 2], [1, 2, 1, 1, 1, 2], [1, 2, 2, 2, 2, 2]],
 [[1, 2, 3, 4, 3, 2], [3, 2, 2, 1, 1, 2], [0, 0, 0, 0, 3, 2]]]

we can now send our input to network, and check the output:

In [ ]:
output

<tf.Tensor 'rnn_2/transpose_1:0' shape=(?, ?, 4) dtype=float32>

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(output, feed_dict={data: sample_input})

array([[[ 1.4933215e-05, -2.8891234e-05, -2.5757893e-06, -1.3985584e-05],
        [ 6.5648273e-05, -1.1769182e-04,  1.1511142e-05, -8.3084160e-05],
        [ 1.8718845e-04, -3.3438808e-04,  4.1369731e-05, -2.4487087e-04]],

       [[ 1.4933215e-05, -2.8891234e-05, -2.5757893e-06, -1.3985584e-05],
        [ 9.4816744e-05, -1.7333034e-04, -3.4586723e-05, -5.4052216e-05],
        [ 2.2284915e-04, -4.2620383e-04, -8.8944653e-05, -1.3436386e-04]]],
      dtype=float32)

As you see, the output is of shape (2, 3, 5), which corresponds to our 2 batches, 3 elements in our sequence, and the dimensionality of the output which is 5.